In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
# Loading the data:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Creating validation data:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, test_size = 0.1)

In [5]:
# Creating preprocessing function:
@tf.function
def preprocess(data):
    data = tf.image.random_flip_left_right(data)
    data = tf.image.random_flip_up_down(data)
    out = tf.cast(data, tf.float32)/255.
    return out

In [6]:
# Preprocessing the images:
x_train, x_valid, x_test = preprocess(x_train), preprocess(x_valid), preprocess(x_test)
x_train, x_valid, x_test = x_train[..., tf.newaxis], x_valid[..., tf.newaxis], x_test[..., tf.newaxis]

In [7]:
# Shuffling and creating batches:
train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
valid_set = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(32)
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [8]:
# Creating a subclassing model:
class FashionNet(keras.Model):
    def __init__(self, **kwargs):
        super(FashionNet, self).__init__(**kwargs)
        self.conv1 = keras.layers.Conv2D(64, 7, activation = tf.nn.relu, padding = 'same')
        self.conv2 = keras.layers.Conv2D(128, 3, activation = tf.nn.relu, padding = 'same')
        self.flatten = keras.layers.Flatten()
        self.hidden1 = keras.layers.Dense(128, activation = tf.nn.relu)
        self.hidden2 = keras.layers.Dense(64, activation = tf.nn.relu)
        self.out = keras.layers.Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        return self.out(x)

In [9]:
# Loading the model:
model = FashionNet()

In [10]:
# Creating loss and optimizers:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True)
optimizer = keras.optimizers.SGD()
metrics = keras.metrics.SparseCategoricalAccuracy()

In [11]:
# Creating a custom training loop:
@tf.function
def train(x_train, y_train, loss_fn = loss, optimizer = optimizer, metrics = metrics, model = model):
    with tf.GradientTape() as tape:
        y_pred = model(x_train, training = True)
        loss = loss_fn(y_train, y_pred)
        metric = metrics(y_train, y_pred)
    grad = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grad, model.trainable_variables))
    return metric, loss

In [12]:
# Creating a training loop:
for epoch in range(7):
    metrics.reset_state()
    for x_train, y_train in train_set:
        metric, losses = train(x_train, y_train)
    
    print(f"Epochs: {epoch + 1}     Loss: {losses}     Accuracy: {metric} ")

Epochs: 1     Loss: 0.43631964921951294     Accuracy: 0.7612037062644958 
Epochs: 2     Loss: 0.3354431986808777     Accuracy: 0.8486666679382324 
Epochs: 3     Loss: 0.246651753783226     Accuracy: 0.8685555458068848 
Epochs: 4     Loss: 0.37773269414901733     Accuracy: 0.8793148398399353 
Epochs: 5     Loss: 0.21964392066001892     Accuracy: 0.8873147964477539 
Epochs: 6     Loss: 0.25400036573410034     Accuracy: 0.8942777514457703 
Epochs: 7     Loss: 0.08701780438423157     Accuracy: 0.9006666541099548 


In [13]:
# Compiling model to evaluate:
model.compile(
    loss = loss,
    optimizer = optimizer,
    metrics = ['accuracy']
)
model.evaluate(test_set)

313/313 [==============================] - 1s 2ms/step - loss: 9.5744 - accuracy: 0.1013


[9.574402809143066, 0.10130000114440918]